In [ ]:
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial.distance import cosine

In [ ]:
class MotionModel:
    def __init__(self, position, friction=0.7):
        assert 0 <= friction <= 1

        self.position = position.copy()
        self.velocity = np.array([0, 0])
        self.friction = friction

    def update(self, position):
        self.velocity = (self.friction * self.velocity +
                         (1 - self.friction) * (position - self.position))
        self.position += self.velocity

def rand_position_step(a=-1, b=1):
    return np.random.random(2) * (b - a) + a

position = np.array([0, 0], dtype=np.float)
model = MotionModel(position)
n_iterations = 10
iterations = np.arange(0, n_iterations)

positions_real = []
positions_model = []

for i in iterations:
    positions_real.append(position.copy())
    positions_model.append(model.position.copy())
    prev_velocity = model.velocity.copy()
    step = rand_position_step()
    position += step
    model.update(position)
    print(f'{prev_velocity} {model.velocity} {cosine(prev_velocity, model.velocity)}')

positions_real = np.array(positions_real)
positions_model = np.array(positions_model)

plt.scatter(positions_real[:, 0], positions_real[:, 1], s=10, label='real')
plt.scatter(positions_model[:, 0], positions_model[:, 1], s=10, label='model')
plt.legend()
plt.show()

In [ ]:
from scipy.spatial.distance import cosine

a = np.array([1, 2])
b = np.array([1, 6])
scale = 0.1
np.tanh(scale * np.linalg.norm(a - b))

In [ ]:
from skimage.metrics import structural_similarity as ssim

img = cv.imread('../data/cars.jpg').astype(np.float)
ssim(img, img + 0.9 * np.random.random(img.shape), multichannel=True)

In [ ]:
import json
import numpy as np
import cv2 as cv
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import backend as K
from tensorflow.keras.applications.mobilenet import preprocess_input

In [ ]:
img_height, img_width, img_depth = 224, 224, 3

if K.image_data_format() == 'channels_first':
    input_shape = (img_depth, img_height, img_width)
else:
    input_shape = (img_height, img_width, img_depth)

def image_show(img):
    cv.imshow('Preview', img)
    cv.waitKey(0)
    cv.destroyAllWindows()

def image_read(img_file_path, width=img_width, height=img_height):
    img = cv.imread(img_file_path, cv.IMREAD_COLOR)
    img = cv.cvtColor(img, cv.COLOR_BGR2RGB)
    return cv.resize(img, (width, height))

img_query = image_read('../data/snapshots/03.png')
img_gt = image_read('../data/snapshots/02.png')

image_show(img_query)
image_show(img_gt)

In [ ]:
with open('../model/embedding_model_layout.json', 'r') as json_file:
    content = json.load(json_file)
    embedding_model = keras.models.model_from_json(json.dumps(content))
embedding_model.load_weights('../model/embedding_model_weights.h5')
embedding_model.summary()

In [ ]:
def get_embedding(img):
    return embedding_model.predict(np.array([preprocess_input(img)]))[0]

query_emb = get_embedding(img_query)
gt_emb = get_embedding(img_gt)

np.linalg.norm(query_emb - gt_emb)

In [3]:
import json

config = {
    'detector': {
        'config_file_path': '../model/yolov3.cfg',
        'weights_file_path': '../model/yolov3.weights',
        'labels_file_path': '../model/coco.names',
        'score_thresh': 0.5,
        'nms_thresh': 0.3,
        'use_gpu': False,
    },
    'tracker': {
        'config_file_path': '../model/embedding_model_layout.json',
        'weights_file_path': '../model/embedding_model_weights.h5',
        'iou_dist_thresh': 0.7,
        'emb_dist_thresh': 0.2,
        'max_no_update_count': 30,
    }
}

with open('../config_files/config_05.json', 'wt') as json_file:
    json_file.write(json.dumps(config, indent=2))